# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [3]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet,self).__init__()
        self.sequential = nn.Sequential(
            nn.Conv2d(1,6,kernel_size = 5, padding = 2),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(2 ,stride = 2),
            nn.Conv2d(6,16,kernel_size = 5),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2 ,stride = 2),
        )
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.cl = nn.Softmax(dim=1)
    
    def forward(self, data):
        out = self.sequential(data)
        out = out.view(out.shape[0],-1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        out = self.cl(out)
        return out

    
model = SimpleNet()

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

Next, we can start to train and evaluate!

In [4]:
import numpy as np
# train and evaluate
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

for epoch in range(NUM_EPOCHS):
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        images.to(device)
        labels.to(device)
        optimizer.zero_grad()
        outputs = model.forward(images)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        
# evaluate
train_acc = 0
test_acc = 0
count = 0
# TODO:calculate the accuracy using traning and testing dataset
torch.no_grad()
for images, labels in tqdm(train_loader):
    count = count + 1
    images.to(device)
    outputs = model.forward(images).detach().cpu().numpy()
    prediction = np.argmax(outputs, axis=1)
    labels = labels.detach().cpu().numpy()
    train_acc += np.mean(prediction==labels)
train_acc = train_acc/count

count = 0
torch.no_grad()
for images, labels in tqdm(test_loader):
    count = count + 1
    images.to(device)
    outputs = model.forward(images).detach().cpu().numpy()
    labels = labels.detach().cpu().numpy()
    prediction = np.argmax(outputs, axis=1)
    test_acc += np.mean(prediction==labels)
test_acc = test_acc/count
    


100%|██████████| 78/78 [00:02<00:00, 27.45it/s]


#### Q5:

Please print the training and testing accuracy.

In [5]:
print('Training accuracy: %0.2f%%' % (train_acc*100))
print('Testing accuracy: %0.2f%%' % (test_acc*100))

Training accuracy: 98.49%
Testing accuracy: 98.53%
